<a href="https://colab.research.google.com/github/alanpirotta/freecodecamp_certif/blob/main/fcc_sms_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


##Import libraries and datasets

In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
train_ds = pd.read_table(train_file_path, names=('result','sms'))
test_ds = pd.read_table(test_file_path, names=('result','sms'))

In [ ]:
train_ds

In [ ]:
test_ds.shape

##Checking unbalance:

In [ ]:
train_count = train_ds['result'].value_counts()/train_ds['result'].count()
print(f'Percentage of {train_count.index[0]} in train: {train_count[0]:.2f}\nPercentage of {train_count.index[1]} in train: {train_count[1]:.2f}')
plt.bar(train_count.index, height=train_count)
plt.show()

In [ ]:
test_count = test_ds['result'].value_counts()/test_ds['result'].count()
print(f'Percentage of {test_count.index[0]} in test: {test_count[0]:.2f}\nPercentage of {test_count.index[1]} in test: {test_count[1]:.2f}')
plt.bar(test_count.index, height=test_count)
plt.show()

i think is quite unbalanced, but may be enough to work with. Both dataset has the same unbalance, so it's another reason to continue with this dataset (probably stratified when splitted).

## Feature engineering

###Convert and extract the result feature in binary (0/1)

In [ ]:
train_ds['result'] = np.where(train_ds['result'] == 'spam', 1, 0)
test_ds['result'] = np.where(test_ds['result'] == 'spam', 1, 0)

In [ ]:
train_y = train_ds.pop('result')
test_y = test_ds.pop('result')

###Tokenize the sms column data

####First: i select one sample of the dataset, to check different options.

In [ ]:
sample = train_ds['sms'].iloc[100]
print(sample)
type(sample)

#####Option 1: with nltk library and word_tokenize. It works well.

In [ ]:
pip install --user -U nltk

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize 

In [ ]:
word_tokenize(sample)

#####Option 2: with keras and text_to_word_sequence
It works well too. I like more this approach, i think is better for the contramptions.

In [ ]:
from keras.preprocessing.text import text_to_word_sequence

In [ ]:
text_to_word_sequence(sample)

#####Option 3: Use keras tokenizer.
Apparently returns the same result as option 2

In [ ]:
from keras.preprocessing.text import Tokenizer

In [ ]:
sample_texts = pd.DataFrame([train_ds.iloc[100].values], index=[100], columns=['sms'])

In [ ]:
max_words = 5000
test_tok = Tokenizer(lower=True)

In [ ]:
test_tok.fit_on_texts(sample_texts['sms'])

In [ ]:
test_tok.word_counts

####Second: Tokenize the entire column with option 3

In [ ]:
max_words = 5000
tok = Tokenizer(num_words=max_words, lower=True)

In [ ]:
tok.fit_on_texts(train_ds['sms'])

In [ ]:
tok.document_count

In [ ]:
tok.word_counts

####Third: Create the sparse matrix for the number of words selected

In [ ]:
train_matrix = tok.texts_to_matrix(train_ds['sms'], mode='binary')
test_matrix = tok.texts_to_matrix(test_ds['sms'], mode='binary')

In [ ]:
train_matrix.shape, test_matrix.shape

In [ ]:
unique, counts = np.unique(train_matrix[100], return_counts=True)
dict(zip(unique, counts))

In [ ]:
tok.word_index.items()

####Third-bis: Another way to vectorize the data, with option 1 tokenizer, generating a df to visually check the data

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
vectorizer = CountVectorizer(binary=True, analyzer=word_tokenize, max_features=3000)

In [ ]:
vectors = vectorizer.fit_transform(train_ds['sms'])

In [ ]:
vocabulary = vectorizer.vocabulary_

In [ ]:
sorted(vocabulary.items(), key=lambda item: item[1])

In [ ]:
columns = [token for token, _ in sorted(vocabulary.items(), key=lambda item: item[1])]

In [ ]:
matrix_df = pd.DataFrame(vectors.todense(), columns=columns, index=[i for i in range(0,len(train_ds))])

In [ ]:
matrix_df

i see el lot of tokens that probably aren't significative for the model (like !, #, $, etc). This could be a way to get better results, filtering those in the tokenize step

###Creating the final sparse matrices with alternative 3rd step

In [ ]:
final_vectorizer = CountVectorizer(binary=True, analyzer=word_tokenize, max_features=3000)

In [ ]:
final_vectorizer.fit(train_ds['sms'])

In [ ]:
train_x = final_vectorizer.transform(train_ds['sms'])
test_x = final_vectorizer.transform(test_ds['sms'])

In [ ]:
train_x.shape, train_y.shape

In [ ]:
test_x.shape, test_y.shape

##Create the classification model

### LogisticRegression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
lr = LogisticRegression()

In [ ]:
lr.fit(train_x, train_y)

In [ ]:
train_pred = lr.predict(train_x)
test_pred = lr.predict(test_x) 

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
accuracy_score(train_y, train_pred), accuracy_score(test_y, test_pred)

In [ ]:
print('train data:')
print(classification_report(train_y, train_pred))
print('-----------------------------------------------------')
print('test data:')
print(classification_report(test_y, test_pred))

In [ ]:
train_ds

##Construct Function

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  series_text =pd.Series(pred_text)
  vector_text = final_vectorizer.transform(series_text)
  prediction = list(lr.predict(vector_text))
  if prediction[0] == 1:
    prediction.append("spam")
  else:
    prediction.append("ham")
  return (prediction)

pred_text = "how are you doing today"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
